## EXPERIMENT 1

### MagFace Results

This notebook loads the feature vectors from MagFace and run them through the results metrics:
FNIR, FPIR, FND, FPD, and GARBE

In [1]:
## Load libraries

import numpy as np
import cv2
from matplotlib import pyplot as plt
import torch
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import normalize
import sys
sns.set(style="white") 
%matplotlib inline
sys.path.append('../../utils') 
from MagFace_utils.MagFace_funcs import *

### Load data

In [2]:
a_df = pd.read_csv('../../data/adults_balanced.csv')
a_mates = a_df.groupby("im_id").agg({'im_id': ['count']})
a_enrolled_ids = a_mates[a_mates[('im_id', 'count')] > 1].index
a_nonmated_ids = a_mates[a_mates[('im_id', 'count')] == 1].index



#### Adults data

In [6]:
# Adults data - enrolled
feature_list_adults = '../../data/feat_adults_new.list'


### OBS brug KATS df
#enrolled_image_names_a = df_enr[df_enr.enrolled == "enrolled"].img_name.to_list()
a_df = pd.read_csv('../../data/adults_balanced.csv')
a_mates = a_df.groupby("im_id").agg({'im_id': ['count']})
enrolled_image_names_a = a_mates[a_mates[('im_id', 'count')] > 1].index
non_enrolled_image_names_a = a_mates[a_mates[('im_id', 'count')] == 1].index


image_name_a_enrolled, identity_names_a_enrolled, num_ids_a_enrolled, norm_feature_vectors_a_enrolled = load_enrolled_magface_vectors(feature_list_adults, enrolled_image_names_a, canonical=False, df_c_can=None)

image_name_a_non_enrolled, identity_names_a_non_enrolled, num_ids_a_non_enrolled, norm_feature_vectors_a_non_enrolled = load_magface_vectors(feature_list_adults, canonical=False, df_c_can=None)



#### Children data

In [7]:
# Adults data - enrolled
feature_list_children = '../../data/feat_children.list'

### OBS brug KATS df
#enrolled_image_names_c = df_enr[df_enr.enrolled == "enrolled"].img_name.to_list()
c_df = pd.read_csv('/Users/gabriellakierulff/Desktop/HCAI/speciale/Master_Thesis/data/child_balanced.csv')
c_mates = c_df.groupby("im_id").agg({'im_id': ['count']})
enrolled_image_names_c = c_mates[c_mates[('im_id', 'count')] > 1].index
non_enrolled_image_names_c = c_mates[c_mates[('im_id', 'count')] == 1].index

image_name_c_enrolled, identity_names_c_enrolled, num_ids_c_enrolled, norm_feature_vectors_c_enrolled = load_enrolled_magface_vectors(feature_list_children, enrolled_image_names_c, canonical=False, df_c_can=None)

image_name_c_non_enrolled, identity_names_c_non_enrolled, num_ids_c_non_enrolled, norm_feature_vectors_c_non_enrolled = load_magface_vectors(feature_list_children, canonical=False, df_c_can=None)



##### Canonical children data

In [ ]:
c_df_can = pd.read_csv("../data/OFIQ_results/canonical_children.csv", sep=";")
#enrolled_image_names_c = df_enr[df_enr.enrolled == "enrolled"].img_name.to_list()
image_name_c_enrolled_can, identity_names_c_enrolled_can, num_ids_c_enrolled_can, norm_feature_vectors_c_enrolled_can = load_enrolled_magface_vectors(feature_list_children, enrolled_image_names_c, canonical=True, df_c_can=None)

image_name_c_non_enrolled_can, identity_names_c_non_enrolled_can, num_ids_c_non_enrolled_can, norm_feature_vectors_c_non_enrolled_can = load_magface_vectors(feature_list_children, canonical=True, df_c_can=None)


## Experiment 1.1

### Similarity matrices
(using nomalized feature vectors from loading above)